## Code build and train ANN

In [ ]:
include("./code/general/utils_general.jl")

include("./code/ann/build_train.jl")

using Pkg; Pkg.add("DelimitedFiles");    
using DelimitedFiles

using Pkg; Pkg.add("Plots")
using Plots; Pkg.add("PyPlot")



#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, validationIndexes, testIndexes) = holdOut(datasetLength, 0.2, 0.2)
(trainDataset, validationDataset, testDataset) = (dataset[trainIndexes, :], dataset[validationIndexes, :], dataset[testIndexes, :])


(trainInput, validationInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, validationDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))


classes = unique(trainDataset[:,5])
(trainOutput, validationOutput, testOutput) = (oneHotEncoding(trainDataset[:,5], classes), oneHotEncoding(validationDataset[:,5], classes), oneHotEncoding(testDataset[:,5], classes))


# Normalization 
normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, validationInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(validationInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))


#Define the parameters
topology = [4, 3]; 
learningRate = 0.01;
numMaxEpochs = 1000; 
topologies_to_try = [[5], [10], [4, 3], [8, 4]]
    
for topo in topologies_to_try
    println("Training with topology: ", topo)
    (ann, trainingLosses, validationLosses, testLosses) = trainClassANN(topo, (trainInputNormalized, trainOutput); validationDataset=(validationInputNormalized, validationOutput), testDataset=(testInputNormalized, testOutput), maxEpochs=numMaxEpochs, learningRate=learningRate);

  g =  plot(1:length(trainingLosses), 
    trainingLosses, 
    label="Train losses", 
    xlabel="Epoch", 
    ylabel="Loss value", 
    title="Loss evolution")

    if !isempty(validationLosses)
        plot!(1:length(validationLosses), validationLosses, label="Validation losses")
    end

    if !isempty(testLosses)
        plot!(1:length(testLosses), testLosses, label="Test losses")
    end

    display(g)
end;  


## Code metrics

In [ ]:
include("./code/general/train_metrics.jl")

#Test boolean and boolean function
outputs=[0,0,1,1,0,1,1,0,0,0,1,1]
targets=[0,1,1,0,1,1,1,0,1,1,0,0]
printConfusionMatrix(convert(Array{Bool,1}, outputs), convert(Array{Bool,1}, targets));
println("\n\n")

#Test real and boolean
outputs=[0.4; 0.6; 1; 0.4;  0.55]
targets=[0;   1;   1; 1; 0]
printConfusionMatrix(convert(Array{Real,1}, outputs), convert(Array{Bool,1}, targets));


#Test array bool
testOutputs=Bool[0 0 1;1 0 0;0 1 0; 0 0 1; 0 0 1]
testTargets=Bool[0 0 1;0 1 0;0 1 0; 1 0 0; 1 0 0]
printConfusionMatrix(testOutputs, testTargets, weighted=false)
println("\n\n")

#Test array real and target bool
testRealOutputs=Real[0 0 0.7;0 0 0.6;0 1 0; 0 0 1; 0 0 1]
printConfusionMatrix(testRealOutputs, testTargets, weighted=false)
println("\n\n")

#Test array any 
result = confusionMatrix(testOutputs, testTargets)

testClasses=["dog", "cat", 3, :green, "dog", 3, "dog"]
testClassesTargets=["dog", "cat", 3, :green, "dog", 3, "dog"]
uniqueClasss=unique(testClasses)
printConfusionMatrix(testClasses, testClassesTargets, uniqueClasss)
println("\n\n")

#Test two classes 
result = confusionMatrix(testOutputs, testTargets)

testClasses=["dog"; "cat"]
testClassesTargets=["dog"; "cat"]
uniqueClasss=unique(testClasses)
printConfusionMatrix(testClasses, testClassesTargets, uniqueClasss)
println("\n\n")
println("hola")

## BUILD AND TRAIN CROSS VALIDATION ANN

In [ ]:
using Pkg; Pkg.add("DelimitedFiles");    
using DelimitedFiles

include("./code/general/utils_general.jl")
include("./code/ann/build_train.jl")


a=[false,true, false, true, true, false]


result = crossvalidation(5, 3)

println("Result", result)

b = crossvalidation(a, 3)
println("Cross validation AbstractArray{Bool, 1}", b)


a = Bool[0 1;1 0;1 0; 0 1;0 1]

bool2_indices = crossvalidation(a, 10)
println("Cross validation AbstractArray{Bool,2} ", bool2_indices)

unique(bool2_indices)


any_array = ['a', :b, 1, 'a']
one = oneHotEncoding(any_array)

println("Size ", size(any_array, 1), " lenght one " ,length(one), " size one ", size(one), "number columns one", size(one, 2))

any_indices = crossvalidation(any_array, 2)
println("Cross validation AbstractArray{<:Any,2} ", any_indices)


#HOW TO

b = [1 1 1 "perro"; 0 0 0 "gato"]


#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, testIndexes) = holdOut(datasetLength, 0.2)
(trainDataset, testDataset) = (dataset[trainIndexes, :], dataset[testIndexes, :])
(trainInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))



classes = unique(trainDataset[:,5])
normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))

topology = [4, 3]; 
crossValidationIndices = crossvalidation(trainDataset[:, 5], 8)


ANNCrossValidation(topology,
        (trainInputNormalized, trainDataset[:, 5]),
        crossValidationIndices)

## MLJ CROSS VALIDATION

In [4]:
import Pkg;
Pkg.add("DelimitedFiles");    
using DelimitedFiles

include("./code/general/model_factory.jl")
include("./code/general/utils_general.jl")



topology = [4, 3]

ann_hyperparameters = Dict(
    :modelType => :ANN,
    :topology => topology,                      
    :transferFunctions => fill(σ, length(topology)),            
    :learningRate => 0.01,                  
    :validationRatio => 0.2,                  
    :maxEpochs => 1000,          
    :maxEpochsVal => 20                         
)


svc_hyperparameters = Dict(
    :modelType => :SVC,
    :cost   => Float64(1.0),                          
    :kernel => "rbf",              
    :degree => Int32(3),                             
    :gamma  => Float64(0.1),                          
    :coef0  => Float64(0.0)                         
)

dt_hyperparameters = Dict(
    :modelType => :DecisionTreeClassifier,
    :max_depth => 5,                                     
    :rng       => Random.MersenneTwister(1)             
)

knn_hyperparameters = Dict(
    :modelType => :KNeighborsClassifier,
    :n_neighbors => 3   
)


#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, testIndexes) = holdOut(datasetLength, 0.2)
(trainDataset, testDataset) = (dataset[trainIndexes, :], dataset[testIndexes, :])
(trainInput, testInput) =  (trainDataset[:,1:4], testDataset[:,1:4])

(trainInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))

normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))

crossValidationIndices = crossvalidation(trainDataset[:, 5], 5)


result = modelCrossValidation(
    :ANN, ann_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result ANN",result)



result = modelCrossValidation(
    :SVC, svc_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result SVC",result)

result = modelCrossValidation(
    :DecisionTreeClassifier, svc_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result DecisionTreeClassifier",result)


result = modelCrossValidation(
    :KNeighborsClassifier, knn_hyperparameters,
    (trainInputNormalized, trainDataset[:, 5]),
    crossValidationIndices)

println("Result KNN",result)

   Resolving package versions...


  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


   Resolving package versions...


...

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Pablo\.julia\packages\MLJModels\ziReN\src\loading.jl:159


   Resolving package versions...


  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


fold 1 de 5 para el modelo SVC


fold 2 de 5 para el modelo SVC
fold 3 de 5 para el modelo SVC
fold 4 de 5 para el modelo SVC
fold 5 de 5 para el modelo SVC
Result SVC((0.9000535117056856, 0.037695925019242384), (0.09994648829431438, 0.037695925019242384), (0.9143866423431641, 0.0321012818509816), (0.9576224692658263, 0.01556231812669208), (0.9000535117056856, 0.037695925019242384), (0.9478876647649027, 0.019459218990471306), (0.8972265510357603, 0.03990027895738469), [37.0 3.0 0.0; 3.0 35.0 2.0; 1.0 3.0 36.0])
fold 1 de 5 para el modelo DecisionTreeClassifier
fold 2 de 5 para el modelo DecisionTreeClassifier
fold 3 de 5 para el modelo DecisionTreeClassifier
fold 4 de 5 para el modelo DecisionTreeClassifier
fold 5 de 5 para el modelo DecisionTreeClassifier
Result DecisionTreeClassifier((0.9512240802675584, 0.03241306207021203), (0.04877591973244144, 0.03241306207021203), (0.9591536502145198, 0.025991280645282333), (0.9786850285264608, 0.01513379467942184), (0.9512240802675584, 0.03241306207021203), (0.9750504295363631

## ENSEMBLE MODELS

In [9]:
using Pkg;
Pkg.add([  
    "UrlDownload",        
    "DataFrames",       
])

using UrlDownload
using DataFrames

include("./code/general/model_factory.jl")
include("./code/general/utils_general.jl")



 
svc_hyperparameters = Dict(
    :modelType => :SVCProbabilistic,
    :cost   => Float64(1.0),                          
    :kernel => "rbf",              
    :degree => Int32(3),                             
    :gamma  => Float64(0.1),                          
    :coef0  => Float64(0.0)                         
)


ensemble_hyperparameters = Dict(
    :n => :10,
    :baggin_fraction => 0.50,                          
    :rng => 123                     
)



url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
data = urldownload(url, true, format=:CSV, header=false) |> DataFrame
insertcols!(data, :Mine => data[:, 61].=="M")
input_data = Matrix(data[!, 1:59]);
output_data = data[!, end];

crossValidationIndices = crossvalidation(output_data, 5)


#HOMOGENEOUS

trainingDataset = (input_data, output_data)
trainClassEnsemble(:SVCProbabilistic, svc_hyperparameters, ensemble_hyperparameters, trainingDataset, crossValidationIndices)

#HETEROGEOUS
trainingDataset = (input_data, output_data)

models = [:SVCProbabilistic, :DecisionTreeClassifier]

modelsHyperParameters = Dict(
    :modelType => :SVC,
    :cost   => Float64(1.0),                          
    :kernel => "rbf",              
    :degree => Int32(3),                             
    :gamma  => Float64(0.1),                          
    :coef0  => Float64(0.0),
    :max_depth =>  5                         
)


ensemble_hyperparameters = Dict(
    :n => :10,
    :baggin_fraction => 0.50,                          
    :rng => 123                     
)


trainClassEnsemble(models, modelsHyperParameters, ensemble_hyperparameters, trainingDataset, crossValidationIndices)

   Resolving package versions...


  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


   Resolving package versions...


  No Changes

 to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


...

Evaluating over 10 metamodels:  50%[============>            ]  ETA: 0:00:01

Evaluating over 10 metamodels:  60%[===============>         ]  ETA: 0:00:01

Evaluating over 10 metamodels:  70%[=================>       ]  ETA: 0:00:00

Evaluating over 10 metamodels:  80%[====================>    ]  ETA: 0:00:00

Evaluating over 10 metamodels:  90%[======================>  ]  ETA: 0:00:00